In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import os
import sys
import math
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib import cm, colors
import copy
import numpy as np
import seaborn as sns
from scipy.spatial import distance_matrix
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from docplex.mp.progress import *
%matplotlib inline
palette = sns.color_palette("bright", 10)

In [2]:
#n=500
#sampled_df1=pd.read_csv("dataset/sub_data_500_20.csv")
sampled_df1=pd.read_csv("C:/Users/Zhifeng Guo/python_code/paper3_code/Lagrange_Relaxation/LR_equality/multiple_randomized_datasets_2000points/dataset/sub_data_2000_1.csv")
n=sampled_df1.shape[0]
df_X=sampled_df1.loc[:, sampled_df1.columns.str.contains('X')]
#df_X=sampled_df1.loc[:, sampled_df1.columns.str.contains('X')]

In [437]:
k0=30
#3 to 17
#k=9
kmeans = KMeans(n_clusters=k0, random_state=0).fit(df_X)

In [438]:
#dfij=pd.DataFrame(distance_matrix(df_X_1.values, df_X_1.values), index=df_X_1.index, columns=df_X_1.index)

In [439]:
#dfij.tail(n=31)

In [440]:
cluster_centers=pd.DataFrame(kmeans.cluster_centers_,columns=df_X.columns, index=["center"+str(i) for i in range(k0)])
frames = [df_X, cluster_centers]
df_X_1 = pd.concat(frames)
dfij=pd.DataFrame(distance_matrix(df_X_1.values, df_X_1.values), index=df_X_1.index, columns=df_X_1.index)
dij=dfij.to_numpy()
dij = np.round(dij, decimals=2)

In [441]:
def find_most_closed_points(dij,k0):
    N=dij.shape[0]
    min_index=[]
    for i in range(N-k0,N):
        mini_value=math.inf
        for j in range(N-k0):
            if dij[i,j]<mini_value:
                mini_value=dij[i,j]
                mini_index=j
        min_index.append(mini_index)
    return min_index

In [442]:
M=range(len(set(find_most_closed_points(dij,k0))))
N=range(sampled_df1.shape[0])
column1=list(set(find_most_closed_points(dij,k0)))

In [443]:
len(find_most_closed_points(dij,k0))

30

In [444]:
pd.DataFrame(column1).to_csv('results/potential_500_20_30_cluster_points.csv')

In [445]:
len(column1)

29

In [446]:
def label_transform(x):
    if x in (1, 2, 3):
        return 'H'
    if x in (52, 53, 54):
        return 'L'
    return 'O'

def label_to_color(x):
    if x in (1,2,3):
        return c4
    if x in (52,53,54):
        return c1
    return c5

def label_to_alphas(x):
    if x in (1,2,3):
        return 1
    if x in (52,53,54):
        return 1
    return 0.4

In [447]:
sampled_df1['al'] = sampled_df1['ACORN_Type'].apply(lambda x:label_transform(x))
H=sampled_df1[sampled_df1['al']=='H'].index
L=sampled_df1[sampled_df1['al']=='L'].index
O=sampled_df1[sampled_df1['al']=='O'].index

In [448]:
ddfij=dfij.iloc[0:n,column1]
ddfij=ddfij.to_numpy()
ddfij = np.round(ddfij, decimals=2)

In [449]:
k_list=[]
times=[]
objective_value=[]

In [450]:
def get_y_values(y):
    Yj=[]
    for i in M:
        if y[i].solution_value==1:
            Yj.append(i)
    return Yj

In [451]:
def LR(name,N,M,H,L,k,ddfij):
    mdl=Model(name=name)
        #set variables
    x=mdl.binary_var_dict([(i,j) for i in N for j in M],lb=0.0, name="x")
    y=mdl.binary_var_dict([j for j in M],name="y")
    mdl.add_constraints(x[i, j] -y[j]<= 0 for i in N for j in M)
    mdl.add_constraint(mdl.sum(y[j] for j in M) ==k)
    mdl.add_constraints(mdl.sum(x[i, j] for j in M) == 1 for i in N)
    mdl.add_constraints(x[i1, j]+x[i2, j] -1<= 0 for i1 in H for i2 in L for j in M)
    mdl.minimize(mdl.sum(ddfij[i, j]*x[i,j] for i in N for j in M))
    sol_model = mdl.solve(clean_before_solve=True)
    return x, y, sol_model

In [452]:
for k in range(3,18):
    start2=datetime.now()
    x, y, sol_model=LR(name="LR",N=N,M=M,H=H,L=L,k=k,ddfij=ddfij)
    running_time=datetime.now()-start2
    cluster_points=get_y_values(y=y)
    pd.DataFrame(cluster_points, columns=['cluster_points']).to_csv('results/with_reduction_500_20_cluster_points_'+str(k)+'_.csv')
    obj=sol_model.get_objective_value()
    objective_value.append(obj)
    times.append(running_time)
    k_list.append(k)
    print(running_time)

0:00:01.900640
0:00:01.999618
0:00:01.787967
0:00:01.517450
0:00:01.691842
0:00:01.666269
0:00:01.421100
0:00:01.597441
0:00:01.497365
0:00:01.437643
0:00:01.408237
0:00:01.550136
0:00:01.461998
0:00:01.433869
0:00:01.596613


In [453]:
pd.DataFrame(list(zip(k_list, times, objective_value)),
              columns=['k','Times','Objective_value']).to_csv('results/times_objective_value_with_reduction_500_20.csv')